# Anotación de entidades mencionadas: Trafilatura, spaCy e LabelStudio

## Trafilatura

Ferramenta deseñada para obter datos textuais a partir da web, e que se pode usar como programa desde a liña de comandos ou como módulo de Python.

Instalación de Trafilatura usando `pip`.

In [ ]:
pip install trafilatura

A opción `-h` proporciona axuda.

In [ ]:
!trafilatura -h

No ficheiro `list.txt` temos a lista das URLs que queremos descargar en formato texto para construír o noso corpus. Se a lista é grande, convén explorar as opcións de paralelización (`--parallel`). Trafilatura pode tamén actuar como un *web crawler*, obtendo sitios web completos ao seguir as ligazóns de forma recursiva (opcións de navegación: `--feed`, `--sitemap`, `--crawl`, etc).

In [ ]:
# Só desde Google Colab!
!wget https://raw.githubusercontent.com/sdocio/curso-veran-2024/main/list.txt -O list.txt

In [ ]:
!cat list.txt

Iniciamos o programa indicándolle que preferimos texto limpo (`--precision`), non queremos comentarios (`--no-comments`) ou tabelas (`--no-tables`).

A lista de URLs para extraer está no ficheiro `list.txt` (opción `-i`) e o resultado vai ser almacenado en `texts` (opción `-o`).

In [ ]:
!trafilatura --no-comments --no-tables --precision -i list.txt -o texts

Comprobamos que hai tres documentos limpos (sen etiquetas HTML) correspondentes ás entradas do ficheiro `list.txt`.

In [ ]:
!ls texts

In [ ]:
!head -3 texts/Di5K6oaTxVWyNSlB.txt

Máis información:

- https://trafilatura.readthedocs.io/en/latest/tutorials.html

## spaCy

spaCy é un módulo ou biblioteca de Python para procesamento de linguaxe natural e soporte para máis de 64 linguas. Inclúe componentes como o recoñecemento de entidades mencionadas, etiquetación morfosintáctica, análise de dependencias sintácticas, segmentación de frases, clasificación de textos ou lematización, entre outros.

![spaCy pipeline](https://spacy.io/images/pipeline.svg "spaCy pipeline")

URL: https://spacy.io/

### Instalación de spaCy

In [ ]:
pip install spacy

### Instalación dun modelo

Mediante `spacy download` descargamos un modelo de tamaño medio para inglés. Nun ambiente real de produción (e non de aprendizaxe ou mostra), a mellor opción sería descargar e usar un modelo maior, como `en_core_web_trf` (baseado en Transformers). Na cela, esta opción fica comentada para que sirva de referencia.

In [ ]:
# !python -m spacy download en_core_web_trf

!python -m spacy download en_core_web_md

### Carga do módulo e o modelo

Cargamos o módulo (`import spacy`) e o modelo que queremos usar. Neste caso os textos están en inglés, polo que usaremos un modelo de tamaño medio (`md`) nesta lingua. Os modelos que apresentan un maior rendemento son os baseados en *Transformers* (`trf`), mais tamén son os que requiren máis recursos de cómputo e almacenamento.

Máis modelos en: https://spacy.io/models

In [ ]:
import spacy

# nlp = spacy.load("en_core_web_trf")

nlp = spacy.load("en_core_web_md")


Lanzamos a *pipeline* de procesamento de spaCy chamando a `nlp()` e usando un pequeno texto de mostra.

In [ ]:
sample = "Google was founded by scientists Larry Page and Sergey Brin while they were students at Stanford University, in California."

doc = nlp(sample)

Unha vez lanzada  a *pipeline*, en `doc` temos os resultados de todos os seus componentes.

In [ ]:
# Componentes da pipeline

nlp.pipe_names

In [ ]:
# PoS tagging (tagger)
for token in doc:
    print(f'{token.text} {token.lemma_} {token.pos_}')

In [ ]:
# Dependency parsing (parser)
for token in doc:
    print(f'{token.text} {token.lemma_} {token.pos_} {token.dep_}')

In [ ]:
# Named-Entities Recognition and Classification (NERC)
for ent in doc.ents:
    print(f'{ent.text} {ent.label_}')

Algúns dos componentes (como o *parser* ou o *NERC*) inclúen un módulo de visualización.

Exemplo de visualización do analizador sintáctico:

In [ ]:
from spacy import displacy

small_doc = nlp("Google was founded by Larry Page and Sergey Brin.")
displacy.render(small_doc, jupyter=True, style="dep", options={"distance": 150})

### Procesado do corpus

Utilizaremos spaCy para procesar os textos do corpus e extraer as entidades mencionadas. A saída será un ficheiro JSON cunha estrutura que nos permita cargalo no editor Label Studio para unha corrección manual posterior dos erros do modelo.

Función que devolve un JSON estruturado para Label Studio a partir da *pipeline* de procesamento de spaCy.

In [ ]:
from itertools import groupby

def doc_to_spans(doc):
    tokens = [(tok.text, tok.idx, tok.ent_type_) for tok in doc]
    results = []
    entities = set()
    for entity, group in groupby(tokens, key=lambda t: t[-1]):
        if not entity:
            continue
        group = list(group)
        _, start, _ = group[0]
        word, last, _ = group[-1]
        text = ' '.join(item[0] for item in group)
        end = last + len(word)
        results.append({
            'from_name': 'label',
            'to_name': 'text',
            'type': 'labels',
            'value': {
                'start': start,
                'end': end,
                'text': text,
                'labels': [entity]
            }
        })
        entities.add(entity)

    return results, entities

Procesado dos textos e almacenamento dos ficheiros JSON.

In [ ]:
import glob
import json
import spacy

# model = "en_core_web_trf"
model = "en_core_web_md"

nlp = spacy.load(model, disable=["tagger", "parser", "attribute_ruler", "lemmatizer"])
labels = set()

for input_file in glob.glob('texts/*.txt'):
    with open(input_file) as ifile:
        output_file = input_file[:-3] + 'json'
        with open(output_file, 'w') as ofile:
            text = ifile.read()
            predictions = []
            tasks = []
            doc = nlp(text)
            results, ents = doc_to_spans(doc)
            tasks.append({'data': {'text': text}, 'predictions': [{'result': results}]})
            labels.update(ents)
            json.dump(tasks, ofile, indent=2)

print("Labels found:")
for label in labels:
    print(label)

In [ ]:
!ls texts

In [ ]:
!head -20 texts/82KWPxwfRjzyVB4l.json

Podemos usar displaCy para mostrar as entidades detectadas. Por exemplo, as do último documento procesado.

In [ ]:
options = {"colors": {"DATE": "lightgreen"}}
displacy.render(doc, jupyter=True, style="ent", options=options)

Para poder corrixir a anotación automática realizada por spaCy, descargamos os ficheiros JSON xerados para poder editalos a continuación en Label Studio.

## Label Studio

Label Studio é unha ferramenta flexíbel de anotación focada para a preparación de datos para adestrar modelos de aprendizaxe automática.

URL: https://labelstud.io/

### Instalación

Instalación en local.

1. Instalación de Miniconda (https://docs.anaconda.com/miniconda/miniconda-other-installer-links/)
2. [Windows] Abrir Miniconda Powershell / [Linux/Mac] Abrir terminal
4. Crear e activar ambiente virtual

`conda create -n curso`  
`conda activate curso`  
`conda install psycopg2`

4. Instalar Label Studio

`pip install label-studio`

5. Abrir o editor

`label-studio`


6. Crear unha conta (sign-up)
7. Crear un proxecto

- Nome do proxecto (*project name*)
- Labelling setup: escoller Natural Language Processing, Named-Entity Recognition
- Adicionar etiquetas  
  No apartado de *labels*, eliminar as que hai e adicionar o listado que obtivemos no paso de xeración dos ficheiros JSON anotados.
- Gardar o proxecto con *save*
- Importar os documentos JSON xerados no apartado anterior